Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import integrate, interpolate
from synthetic import ODEModelBaseClass, gillespie_noise_raw

Define classes & functions.  Architecture based on `synthetic.py`, equations based on Rocsoreanu et al. (2000)

In [ ]:
class FitzHughNagumoModel(ODEModelBaseClass):
    # Defined according to Rocsoreanu et al. (2000), eqn 1.1.17
    # VARIABLES
    # x :: membrane voltage
    # y :: linear recovery variable
    # PARAMETERS
    # a ::
    # b ::
    # c ::
    def odes(self, var_array, timeaxis, *args):
        # Define variables
        x = var_array[0]
        y = var_array[1]
        # Define parameters
        a = args[0]
        b = args[1]
        c = args[2]
        # Define differential equations
        dx_dt = c*(x + y - (x**3)/3)
        dy_dt = (-1/c)*(x - a + b*y)
        return [dx_dt, dy_dt]
    
class FitzHughNagumoModelStochastic(FitzHughNagumoModel):
    def __init__(self, timeaxis, init_cond, ode_parameters):
        super().__init__(timeaxis, init_cond, ode_parameters)

        self.ode_parameters_original = self.ode_parameters

        # a
        self.a_array = self.ode_parameters_original["a"]
        self.a_interp1d = interpolate.interp1d(self.timeaxis, self.a_array)
        self.ode_parameters["a"] = self.a

        # b
        self.b_array = self.ode_parameters_original["b"]
        self.b_interp1d = interpolate.interp1d(self.timeaxis, self.b_array)
        self.ode_parameters["b"] = self.b
        
        # c
        self.c_array = self.ode_parameters_original["c"]
        self.c_interp1d = interpolate.interp1d(self.timeaxis, self.c_array)
        self.ode_parameters["c"] = self.c

    # Assumes timeaxis and ext_stimulus_array have the same number of elements
    def a(self, t):
        if t > np.max(self.timeaxis):
            t = np.max(self.timeaxis)
        return self.a_interp1d(np.asarray(t))

    def b(self, t):
        if t > np.max(self.timeaxis):
            t = np.max(self.timeaxis)
        return self.b_interp1d(np.asarray(t))
    
    def c(self, t):
        if t > np.max(self.timeaxis):
            t = np.max(self.timeaxis)
        return self.c_interp1d(np.asarray(t))

    def odes(self, var_array, t, *args):
        # Define variables
        x = var_array[0]
        y = var_array[1]
        # Define parameters
        a = args[0]
        b = args[1]
        c = args[2]
        # Define differential equations
        dx_dt = c(t)*(x + y - (x**3)/3)
        dy_dt = (-1/c(t))*(x - a(t) + b(t)*y)
        return [dx_dt, dy_dt]

    
def fitzhugh_nagumo(
    timeaxis=np.linspace(0, 1000, 1000),
    voltage_init=0,
    recovery_init=0,
    a=1,
    b=1,
    c=1,
):
    model = FitzHughNagumoModel(
        timeaxis=timeaxis,
        init_cond={"x": voltage_init, "y": recovery_init},
        ode_parameters={"a": a, "b": b, "c": c},
    )
    var_out = model.solver()
    voltage = var_out.T[0]
    recovery = var_out.T[1]
    return voltage, recovery

def fitzhugh_nagumo_stochastic(
    timeaxis=np.linspace(0, 1000, 1000),
    voltage_init=0,
    recovery_init=0,
    a_array=0.05 * np.random.rand(1000) + 1,
    b_array=0.05 * np.random.rand(1000) + 1,
    c_array=0.05 * np.random.rand(1000) + 1,
):
    model = FitzHughNagumoModelStochastic(
        timeaxis=timeaxis,
        init_cond={"x": voltage_init, "y": recovery_init},
        ode_parameters={
            "a": a_array,
            "b": b_array,
            "c": c_array,
        },
    )
    var_out = model.solver()
    voltage = var_out.T[0]
    recovery = var_out.T[1]
    return voltage, recovery

Deterministic

In [ ]:
v, r = fitzhugh_nagumo(
    timeaxis=np.linspace(0, 200, 200),
    a=0.994, b=0, c=5)

In [ ]:
plt.subplots(figsize=(20,5))
plt.plot(v)

Stochastic - white noise

In [ ]:
a_array=0.002 * np.random.rand(200) + 0.994
b_array=0.00 * np.random.rand(200) + 0
c_array=0.00 * np.random.rand(200) + 5

vs, rs = fitzhugh_nagumo_stochastic(
    timeaxis=np.linspace(0, 200, 200),
    a_array=a_array, b_array=b_array, c_array=c_array
)

In [ ]:
plt.subplots(figsize=(20,5))
plt.plot(vs)

plt.subplots(figsize=(20,5))
plt.plot(a_array)

In [ ]:
plt.subplots(figsize=(20,5))
plt.plot(vs)

plt.subplots(figsize=(20,5))
plt.plot(a_array)

> Conclusion: As expected, if the stochastic parameter is near a sensitive point and varies a bit, the system alternates between several types of oscillator.  It doesn't lead to as much of a noise in the time series as I expected, but I've come to expect that by now.  The problem now is surveying the parameter space to find other parts that give rise to behaviour like this.

Stochastic - Gillespie noise

In [ ]:
timeaxis=np.linspace(0, 200, 200)

In [ ]:
# Generate Gillespie noise (raw)
noise_timescale = 2
noise_amp = 50

gill_time_final = 300
gill_num_intervals = 200
gill_noise_time, gill_noise_list = gillespie_noise_raw(
    num_timeseries=1,
    noise_timescale=noise_timescale,
    noise_amp=noise_amp,
    time_final=gill_time_final,
)

print("Gillespie noise generated.")

# Scale Gillespie time axis to fit time axis
for gill_time_element in gill_noise_time:
    gill_time_element -= gill_time_element[0]
    gill_time_element *= timeaxis[-1] / gill_time_element[-1]

# Define arrays
a = 0.994
std = 0.002

a_array = (gill_noise_list[0] * std) + a
b_array=0.00 * gill_noise_time[0] + 0
c_array=0.00 * gill_noise_time[0] + 5

# Simulate
print("FHN simulation starts.")
vs, rs = fitzhugh_nagumo_stochastic(
    timeaxis=gill_noise_time[0],
    a_array=a_array, b_array=b_array, c_array=c_array
)
print("FHN simulation done.")

In [ ]:
plt.subplots(figsize=(20,5))
plt.plot(gill_noise_time[0], vs)

plt.subplots(figsize=(20,5))
plt.plot(gill_noise_time[0], a_array)

> Seems to transition between more types of behaviour, but this depends on the dynamic range of $a$, which in turn is derived from the parameters.  I don't think I'm near a regime where noise in the parameters translates to noise in the time series, even if I have 7,000+ time points because I'm taking raw Gillespie noise values.